In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [2]:
train_data, train_label = load_wine(return_X_y=True)

#### Keeping only the first two classes to create a binary problem as instructed
train_data = np.array(train_data[:(59+71)])
train_label = np.array(train_label[:(59+71)])

train_data = (train_data - train_data.mean(axis=0))/(train_data.max(axis=0)-train_data.min(axis=0))

In [3]:
logist_regr = LogisticRegression(solver='sag', penalty=None, max_iter=20000).fit(train_data, train_label)
ideal_loss = log_loss(train_label, logist_regr.predict_proba(train_data))

print(ideal_loss)

0.0002595980939053694
